In [1]:
# Hidden cell for imports
from dataclasses import dataclass
import pychor
import galois
from nacl.public import PrivateKey, PublicKey, SealedBox
import numpy as np
from typing import List
GF_2 = galois.GF(2)

p1 = pychor.Party('p1')
p2 = pychor.Party('p2')

from nacl.utils import random

# Protocol for 1-out-of-n OT
def protocol_ot(sender, receiver, inputs, selection, n):
    # Function for the Receiver to generate keys
    @pychor.local_function
    def gen_keys(selection, n):
        # Generate a single real key pair key = (sk, pk)
        key = PrivateKey.generate()
        public_keys = [PublicKey(random(PublicKey.SIZE)) for _ in range(n)]
        public_keys[selection] = key.public_key
        return key, public_keys

    # Function for the Sender to encrypt the secret inputs
    @pychor.local_function
    def encrypt_inputs(pub_keys, inputs):
        # Encode the inputs as bytes
        length = max([(int(x).bit_length() + 7) // 8 for x in inputs])
        inputs_bytes = [int(x).to_bytes(length, 'little') for x in inputs]
    
        # Encrypt the inputs
        encrypted_inputs = [SealedBox(pk).encrypt(x) for pk, x in \
                            zip(pub_keys, inputs_bytes)]
        return encrypted_inputs

    # Function for the Receiver to decrypt the result
    @pychor.local_function
    def decrypt_result(selection, key, encrypted_inputs):
        # Select the correct input
        selected_input = encrypted_inputs[selection]
        # Decrypt it and convert it from bytes to int
        plaintext = SealedBox(key).decrypt(selected_input)
        return int.from_bytes(plaintext, 'little')

    # Step 1: Generate keys and send to Sender
    sk, pub_keys = gen_keys(selection, n).untup(2)
    pub_keys.send(receiver, sender)

    # Step 2: Encrypt inputs and send to Receiver
    encrypted_inputs = encrypt_inputs(pub_keys, inputs)
    encrypted_inputs.send(sender, receiver)

    # Step 3: Decrypt result
    result = decrypt_result(selection, sk, encrypted_inputs)

    return result

@pychor.local_function
def truth_table(a1, b1, c1):
    possible_c2s = []
    # Consider all possibilities for a2 and b2
    for a2 in GF_2([0, 1]):
        for b2 in GF_2([0, 1]):
            # Compute the share c2 in terms of the others
            c2 = (a1+a2) * (b1+b2) - c1
            possible_c2s.append(c2)
    return possible_c2s

def protocol_gmw_mult(x, y):
    # Function for computing the selection index
    @pychor.local_function
    def compute_selection(a2, b2):
        return int(a2)*2 + int(b2)

    x1, x2 = x
    y1, y2 = y
    z1 = p1.constant(GF_2.Random())
    z2 = protocol_ot(sender=p1, receiver=p2, 
                     inputs=truth_table(x1, y1, z1),
                     selection=compute_selection(x2, y2),
                     n=4)
    return z1, pychor.locally(GF_2, z2)

@pychor.local_function
def share(x):
    s1 = GF_2.Random()
    s2 = GF_2(x) - s1
    return s1, s2

@dataclass
class SecBit:
    # s1 is p1's share of the value, and s2 is p2's share
    # all values (and shares) are binary
    s1: GF_2
    s2: GF_2

    @classmethod
    def input(cls, val):
        """Secret share an input: p1 holds s1, and p2 holds s2"""
        s1, s2 = share(val).untup(2)
        if p1 in val.parties:
            s2.send(p1, p2)
            return SecBit(s1, s2)
        else:
            s1.send(p2, p1)
            return SecBit(s1, s2)

    def __add__(x, y):
        """Add two SecBit objects using local addition of shares"""
        return SecBit(x.s1 + y.s1, x.s2 + y.s2)

    def __mul__(x, y):
        """Multiply two SecBit objects using GMW multiplication"""
        r1, r2 = protocol_gmw_mult((x.s1, x.s2), (y.s1, y.s2))
        return SecBit(r1, r2)

    def reveal(self):
        """Reveal the secret value by broadcast and reconstruction"""
        self.s1.send(p1, p2)
        self.s2.send(p2, p1)
        return self.s1 + self.s2

# Applying Binary MPC

Building applications with binary representations is a little more involved than with arithmetic ones, since most applications require more than just operations on a single bit. The most common approach is to adopt binary representations of larger numbers that mimic hardware, and then implement operations on those representations in exactly the same way as they are implemented in hardware - for example, an adder circuit for 32-bit integers.

## `SecBitInt`: Secure Binary Integers

We can define a `SecBitInt` class to represent integers using a *list of `SecBit` objects* implemented via GMW, and define operations like addition and multiplication for this representation. The `input` class method takes an integer as input, and performs a bit decomposition to turn it into a binary representation before secret sharing the bits. The addition and multiplication methods call binary adder and multiplier functions that we'll define later. The `reveal` method reverses the process, turning the list of bits into a Python integer.

In [7]:
@pychor.local_function
def bit_decompose(x, nbits):
    x = int(x)
    return [(x >> i) & 1 for i in range(nbits)]

@pychor.local_function
def bit_compose(bits):
    return sum([int(b) * 2**i for i, b in enumerate(bits)])
    
@dataclass
class SecBitInt:
    bits: List[SecBit]

    @classmethod
    def input(cls, val, num_bits=32):
        bits = bit_decompose(val, num_bits).unlist(num_bits)
        sec_bits = [SecBit.input(b) for b in bits]
        return SecBitInt(sec_bits)

    def __add__(x, y):
        return SecBitInt(adder(x.bits, y.bits))

    def __mul__(x, y):
        return SecBitInt(multiplier(x.bits, y.bits))

    def reveal(self):
        revealed_bits = [b.reveal() for b in self.bits]
        return bit_compose(revealed_bits)

In [8]:
with pychor.LocalBackend():
    x = SecBitInt.input(p1.constant(3), num_bits=3)
    print('Representation of x:', x)
    print('Value of x:', x.reveal())

Representation of x: SecBitInt(bits=[SecBit(s1=1@{p1}, s2=0@{p1, p2}), SecBit(s1=0@{p1}, s2=1@{p1, p2}), SecBit(s1=1@{p1}, s2=1@{p1, p2})])
Value of x: 3@{p1, p2}


To define addition of `SecBitInt` objects, we can implement the same logic as an [adder circuit](https://en.wikipedia.org/wiki/Adder_(electronics)). This approach computes the result bitwise, maintaining a *carry bit* to track overflow from one digit to the next during the addition.

In [9]:
def adder(x, y):
    output = []
    carry = SecBit(p1.constant(GF_2(0)), p2.constant(GF_2(0)))
    for a, b in zip(x, y):
        out = a + b + carry
        output.append(out)
        carry = ((a + carry) * (b + carry) + carry)
    return output

In [10]:
with pychor.LocalBackend():
    x = SecBitInt.input(p1.constant(3), num_bits=3)
    print('x + x:', (x+x).reveal())

x + x: 6@{p1, p2}


If we run out of bits then the results will overflow, just like in hardware.

In [11]:
with pychor.LocalBackend():
    x = SecBitInt.input(p1.constant(3), num_bits=3)
    print('x + x + x:', (x+x+x).reveal())

x + x + x: 1@{p1, p2}


This approach of representing numbers using binary values and performing computations using approaches from hardware circuits is common in real MPC systems. The binary representation is flexible (more on this later), and operations in $GF(2)$ are cheap, so the performance of these systems often appears high when measured with microbenchmarks.

However, each conceptual operation (e.g. "add two numbers") requires potentially many binary operations, depending on the complexity of the circuit used to implement it. For example, our `adder` function performs one multiplication *per bit in the representation*, just like an actual adder circuit. This means that for 32-bit numbers, we need to perform 32 binary multiplications per integer addition - and each multiplication requires one OT. In systems based on arithmetic shares, integer addition is just field addition, and can be performed without any communication at all between the parties! As a result, arithmetic systems are often faster than binary ones, when the arithmetic representation is sufficient for the application.



## Comparison Operators

One advantage of binary representations for integers is flexibility: it's easy to define common comparison operators like equality, less-than, and greater-than. To test whether $x=y$, we can simply compare the corresponding bits of $x$ and $y$ for equality. Two bits $a$ and $b$ are equal if either $a$ and $b$ are both 1, or $a$ and $b$ are both 0; we can write the following expression in $GF(2)$ for this test:

$$
(a * b) + ((a + 1) * (b + 1))
$$

We can add the `__eq__` method to our class to implement equality testing. This method returns a `SecBit` that is 0 if the result is `False` and 1 if the result is `True`.

In [17]:
class SecBitInt(SecBitInt):
    def __eq__(x, y):
        one = SecBit(p1.constant(GF_2(1)), p2.constant(GF_2(0)))
        is_equal = one
        for a, b in zip(x.bits, y.bits):
            is_equal = is_equal * ((a * b) + ((a + one) * (b + one)))
        return is_equal

In [18]:
with pychor.LocalBackend():
    x = SecBitInt.input(p1.constant(3), num_bits=3)
    print('x == x:', (x == x).reveal())
    print('(x+x) == x:', (x + x == x).reveal())

x == x: 1@{p1, p2}
(x+x) == x: 0@{p1, p2}


In [19]:
# TODO: less than, greater than

## Fixed-Point Decimal Numbers and Shifting

TODO